# Hybrid Monte Carlo

## Monte Carlo Simulation

In this notebook we illustrate the Monte Carlo simulation framework. This includes is structured along the following steps:

  1.  Setting up a model
  2.  Contructing and running a simulation
  3.  Inspecting the simulation
  4.  Calculate future modeled quantities

We use a couple of standard packages for calculation and analysis

In [ ]:
import sys
sys.path.append('../')  # make python find our modules
import numpy as np
import pandas as pd
import plotly.express as px

The following auxilliary method lists the relevant members of an object. We use it to inspect the objects created. 

In [ ]:
def members(obj):
    return [f for f in dir(obj) if not f.startswith('_')] 

The starting point for modelling is a discount curve. For now we use a flat forward furve. More advanced curve specifications can easily be included e.g. via QuantLib. Our modelling framework only requires that a yieldcurve provides a method *discunt(t)* for a time parameter *t*.

In [ ]:
from src.termstructures.YieldCurve import YieldCurve
discCurve = YieldCurve(rate=0.03)
display(members(discCurve))

### Setting up a model

For this example we use a Hull White interest rate model. This type of model will typically also be the building block for more complex models.

The Hull White model allows for piece-wise constant short rate volatility and constant mean reversion.

In [ ]:
from src.models.HullWhiteModel import HullWhiteModel
times = np.array([ 2.0,    5.0,    10.0    ])
vols  = np.array([ 0.0050, 0.0060,  0.0070 ])
mean  = 0.03
#
model = HullWhiteModel(discCurve,mean,times,vols)
display(members(model))

### Contructing and running a simulation

For a simulation we need to specify a time grid on which model states are evolved, number of Monte Carlo paths and a (pseudo) random number seed.

The simulation is run at construction of the object.

In [ ]:
from src.simulations.MCSimulation import MCSimulation
seed = 314159265359
nPaths = 3
times = np.linspace(0.0, 10.0, 11)
display(times)

In [ ]:
mcsim = MCSimulation(model,times,nPaths,seed)
display(members(mcsim))

### Inspecting the simulation

The Monte Carlo simulation calculates independent standard normal variables *dW*. The data are stored as 3-dimensional array *dW\[nPaths,times-1,factor\]*.

In [ ]:
display(mcsim.dW.shape)

The Hull White model is a 1-factor model. Thus last dimension is 1.

We illustrate random samples per time step and path.

In [ ]:
# we prep the dW's for nice plotting
data = pd.DataFrame(mcsim.dW[:,:,0]).T
data.columns = [str(c) for c in data.columns]
ts = pd.Series(times[:-1],name='times')
data = pd.concat([ts,data],axis=1)
data = pd.melt(data,id_vars='times', value_vars=list(data.columns[1:]),var_name='path',value_name='dW')
# 
fig = px.scatter(data,x='times',y='dW',color='path')
fig.show()

Based on the model specification and the model's *evolve()* method model states *X* are simulated. This is the core functionality of Monte Carlo simulation. State variables are also stored in a 3-dimensional array *X\[nPaths,times,states\]*.

In [ ]:
display(mcsim.X.shape)

We simulate the Hull White state variable *x(t)=r(t)-f(0,t)*. However, for derivative pricing we also need the numeraire (here bank account), $B(t) = P(0,t)^{-1}\exp\left\{\int_0^t x(s)sd\right\}$.

Thus, we store the integrated state variable $s(t)=\int_0^t x(s)ds$ in the second component of the Monte Carlo state.

In [ ]:
data_x = pd.DataFrame(mcsim.X[:,:,0]).T
data_x.columns = ['x_'+str(c) for c in data_x.columns]
data_s = pd.DataFrame(mcsim.X[:,:,1]).T
data_s.columns = ['s_'+str(c) for c in data_s.columns]
ts = pd.Series(times,name='times')
data = pd.concat([ts,data_x,data_s], axis=1)
data = pd.melt(data,id_vars='times', value_vars=list(data.columns[1:]),var_name='path',value_name='X')
# 
fig = px.scatter(data,x='times',y='X',color='path')
fig.show()

### Calculate future modeled quantities

With the Monte Carlo state(s) we can now calculate simulated future model quantities. For an interest rate model these quantities are zero coupon bonds $P(t,T)=P(x;t,T)$.

In [ ]:
k = 10  # we pick the last simulated time
t = times[k]
T = t + 5.0  # and 5y zero bond maturity
display((t,T))

The simulates Monte Carlo states:

In [ ]:
states = mcsim.X[:,k,:]
display(states)

And resulting simulated zero coupon bonds:

In [ ]:
zeroBonds = np.array([ model.zeroBond(t,T,X,None) for X in states ])
display(zeroBonds)

We can compare simulated zero coupon bonds to forward zero bonds $P(0,T)/P(0,t)$ calculated from the initial discount curve.

In [ ]:
zeroBondFwd = discCurve.discount(T)/discCurve.discount(t)
display(zeroBondFwd)

However, we need to change measure. Hull White model simulation is in risk-neutral measure. Forward zero bond is a martingale in the *T*-forward measure.

In [ ]:
numeraires = np.array([ model.numeraire(t,X) for X in states ])
zeroBonds_T = zeroBonds / numeraires / discCurve.discount(t)
display(zeroBonds_T)

In [ ]:
zeroBonds_T_av = np.average(zeroBonds_T)
display(zeroBonds_T_av)